# Building an LLM Agent to Find Relevant Research Papers from Arxiv
Updated by: Wilfredo Aaron Sosa Ramos
Credits to: Andrei Chernov.

In this tutorial, we will create an LLM agent based on the **Google Generative AI** language model. The agent's primary purpose will be to find and summarize research papers from **Arxiv** that are relevant to the user's query. To build the agent, we will use the **LlamaIndex** framework.

## Tools Used by the Agent

The agent will utilize the following three tools:

1. **RAG Query Engine**
   This tool will store and retrieve recent papers from Arxiv, serving as a knowledge base for efficient and quick access to relevant information.

2. **Paper Fetch Tool**
   If the user specifies a topic that is not covered in the RAG Query Engine, this tool will fetch recent papers on the specified topic directly from Arxiv.

3. **PDF Download Tool**
   This tool allows the agent to download a research paper's PDF file locally using a link provided by Arxiv.

### First, let's install necessary libraries

In [2]:
import os
import shutil

# Path to check
path = '/content/index'

# Check if the file or directory exists
if os.path.exists(path):
    # Remove the file or directory
    if os.path.isfile(path):
        os.remove(path)  # Remove file
        print(f"File '{path}' has been removed.")
    elif os.path.isdir(path):
        shutil.rmtree(path)  # Use shutil.rmtree to remove non-empty directories
        print(f"Directory '{path}' has been removed.")
else:
    print(f"The path '{path}' does not exist.")

Directory '/content/index' has been removed.


In [1]:
!pip install arxiv==2.1.3 llama_index==0.12.3 llama-index-llms-gemini llama-index-embeddings-gemini

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 629.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256

In [3]:
from getpass import getpass
import requests
import sys
import arxiv
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document, StorageContext, load_index_from_storage, PromptTemplate, Settings
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent


### Additionally, You Need to Provide Your API Key

In [4]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [5]:
llm = Gemini(
    model="models/gemini-1.5-flash",
)

### To Build a RAG Query Engine, We Will Need an Embedding Model

For this tutorial, we will use the MistralAI embedding model.

In [6]:
model_name = "models/embedding-001"
api_key = userdata.get('GOOGLE_API_KEY')
embed_model = GeminiEmbedding(model_name=model_name, api_key=api_key)

### Now, We Will Download Recent Papers About Large Language Models from ArXiv

To keep this tutorial accessible with the free Mistral API version, we will download only the last 10 papers. Downloading more would exceed the limit later while building the RAG query engine. However, if you have a Mistral subscription, you can download additional papers.

In [7]:
def fetch_arxiv_papers(title: str, papers_count: int):
    search_query = f'all:"{title}"'
    search = arxiv.Search(
        query=search_query,
        max_results=papers_count,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    papers = []
    # Use the Client for searching
    client = arxiv.Client()

    # Execute the search
    search = client.results(search)

    for result in search:
        paper_info = {
                'title': result.title,
                'authors': [author.name for author in result.authors],
                'summary': result.summary,
                'published': result.published,
                'journal_ref': result.journal_ref,
                'doi': result.doi,
                'primary_category': result.primary_category,
                'categories': result.categories,
                'pdf_url': result.pdf_url,
                'arxiv_url': result.entry_id
            }
        papers.append(paper_info)

    return papers

papers = fetch_arxiv_papers("Language Models", 10)

In [8]:
[[p['title']] for p in papers]

[['HoVLE: Unleashing the Power of Monolithic Vision-Language Models with Holistic Vision-Language Embedding'],
 ['Frequency Is What You Need: Word-frequency Masking Benefits Vision-Language Model Pre-training'],
 ['Offline Reinforcement Learning for LLM Multi-Step Reasoning'],
 ['Can LLMs Obfuscate Code? A Systematic Analysis of Large Language Models into Assembly Code Obfuscation'],
 ['Data-Driven Mechanism Design: Jointly Eliciting Preferences and Information'],
 ['PromptOptMe: Error-Aware Prompt Compression for LLM-based MT Evaluation Metrics'],
 ['Deciphering the Underserved: Benchmarking LLM OCR for Low-Resource Scripts'],
 ['PruneVid: Visual Token Pruning for Efficient Video Large Language Models'],
 ["The Content Moderator's Dilemma: Removal of Toxic Content and Distortions to Online Discourse"],
 ['Demystifying the Potential of ChatGPT-4 Vision for Construction Progress Monitoring']]

### To Build a RAG Agent, We First Need to Index All Documents

This process creates a vector representation for each chunk of a document using the embedding model.

In [9]:
def create_documents_from_papers(papers):
    documents = []
    for paper in papers:
        content = f"Title: {paper['title']}\n" \
                  f"Authors: {', '.join(paper['authors'])}\n" \
                  f"Summary: {paper['summary']}\n" \
                  f"Published: {paper['published']}\n" \
                  f"Journal Reference: {paper['journal_ref']}\n" \
                  f"DOI: {paper['doi']}\n" \
                  f"Primary Category: {paper['primary_category']}\n" \
                  f"Categories: {', '.join(paper['categories'])}\n" \
                  f"PDF URL: {paper['pdf_url']}\n" \
                  f"arXiv URL: {paper['arxiv_url']}\n"
        print(content)
        documents.append(Document(text=content))
    return documents



#Create documents for LlamaIndex
documents = create_documents_from_papers(papers)

Title: HoVLE: Unleashing the Power of Monolithic Vision-Language Models with Holistic Vision-Language Embedding
Authors: Chenxin Tao, Shiqian Su, Xizhou Zhu, Chenyu Zhang, Zhe Chen, Jiawen Liu, Wenhai Wang, Lewei Lu, Gao Huang, Yu Qiao, Jifeng Dai
Summary: The rapid advance of Large Language Models (LLMs) has catalyzed the
development of Vision-Language Models (VLMs). Monolithic VLMs, which avoid
modality-specific encoders, offer a promising alternative to the compositional
ones but face the challenge of inferior performance. Most existing monolithic
VLMs require tuning pre-trained LLMs to acquire vision abilities, which may
degrade their language capabilities. To address this dilemma, this paper
presents a novel high-performance monolithic VLM named HoVLE. We note that LLMs
have been shown capable of interpreting images, when image embeddings are
aligned with text embeddings. The challenge for current monolithic VLMs
actually lies in the lack of a holistic embedding module for both vi

In [10]:
Settings.chunk_size = 1024
Settings.chunk_overlap = 50

index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

### Now, We Will Store the Index

Indexing a large number of texts can be time-consuming and costly since it requires making API calls to the embedding model. In real-world applications, it is better to store the index in a vector database to avoid reindexing. However, for simplicity, we will store the index locally in a directory in this tutorial, without using a vector database.

In [11]:
index.storage_context.persist('index/')
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir='index/')

#load index
index = load_index_from_storage(storage_context, embed_model=embed_model)

### We Are Ready to Build a RAG Query Engine for Our Agent

It is a good practice to provide a meaningful name and a clear description for each tool. This helps the agent select the most appropriate tool when needed.

In [12]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

rag_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="research_paper_query_engine_tool",
    description="A RAG engine with recent research papers.",
)

### Let's Take a Look at the Prompts the RAG Tool Uses to Answer a Query Based on Context

Note that there are two prompts. By default, LlamaIndex uses a refine prompt before returning an answer. You can find more information about the response modes [here](https://docs.llamaindex.ai/en/v0.10.34/module_guides/deploying/query_engine/response_modes/).

In [13]:
from llama_index.core import PromptTemplate
from IPython.display import Markdown, display
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}" f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template**Text:** 

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt Key**: response_synthesizer:refine_template**Text:** 

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


### Building two other tools is straightforward because they are simply Python functions.

In [14]:
def download_pdf(pdf_url, output_file):
    """
    Downloads a PDF file from the given URL and saves it to the specified file.

    Args:
        pdf_url (str): The URL of the PDF file to download.
        output_file (str): The path and name of the file to save the PDF to.

    Returns:
        str: A message indicating success or the nature of an error.
    """
    try:
        # Send a GET request to the PDF URL
        response = requests.get(pdf_url)
        response.raise_for_status()  # Raise an error for HTTP issues

        # Write the content of the PDF to the output file
        with open(output_file, "wb") as file:
            file.write(response.content)

        return f"PDF downloaded successfully and saved as '{output_file}'."

    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

In [15]:
download_pdf_tool = FunctionTool.from_defaults(
    download_pdf,
    name='download_pdf_file_tool',
    description='python function, which downloads a pdf file by link'
)
fetch_arxiv_tool = FunctionTool.from_defaults(
    fetch_arxiv_papers,
    name='fetch_from_arxiv',
    description='download the {max_results} recent papers regarding the topic {title} from arxiv'
)


In [16]:
def present_final_result(papers):
    final_content = []
    for paper in papers:
        content = f"Title: {paper['title']}\n" \
                  f"Authors: {', '.join(paper['authors'])}\n" \
                  f"Summary: {paper['summary']}\n" \
                  f"Published: {paper['published']}\n" \
                  f"Journal Reference: {paper['journal_ref']}\n" \
                  f"DOI: {paper['doi']}\n" \
                  f"Primary Category: {paper['primary_category']}\n" \
                  f"Categories: {', '.join(paper['categories'])}\n" \
                  f"PDF URL: {paper['pdf_url']}\n" \
                  f"arXiv URL: {paper['arxiv_url']}\n"
        final_content.append(content)
    return final_content

In [17]:
present_final_result_tool = FunctionTool.from_defaults(
    download_pdf,
    name='present_final_result',
    description='python function, which formats what information from the papers should be shown and not just mere description'
)

In [18]:
# building an ReAct Agent with the four tools.
agent = ReActAgent.from_tools([download_pdf_tool, rag_tool, fetch_arxiv_tool, present_final_result_tool], llm=llm, verbose=True)

### Let's Chat with Our Agent

We built a ReAct agent, which operates in two main stages:

1. **Reasoning**: Upon receiving a query, the agent evaluates whether it has enough information to answer directly or if it needs to use a tool.
2. **Acting**: If the agent decides to use a tool, it executes the tool and then returns to the Reasoning stage to determine whether it can now answer the query or if further tool usage is necessary.

In [32]:
# create a prompt template to chat with an agent
q_template = (
    "I am interested in {topic}. \n"
    "Find papers in your knowledge database related to this topic; use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to {topic}'. You must use the present_final_result tool for presenting the final results.  If there are not, could you fetch the recent one from arXiv? Use this tool: fetch_from_arxiv \n"
)

In [27]:
answer = agent.chat(q_template.format(topic="Multimodal Large Language Models."))

> Running step 8f5d0360-a721-4462-b068-d4689296a36c. Step input: I am interested in Multimodal Large Language Models.. 
Find papers in your knowledge database related to this topic; use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to Multimodal Large Language Models.'. If there are not, could you fetch the recent one from arXiv? 

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.  I will first try the `research_paper_query_engine_tool` and if it doesn't find sufficient information, I will use `fetch_from_arxiv`.
Action: research_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, authors and link to download for papers related to Multimodal Large Language Models.'}
Observation: **PruneVid: Visual Token Pruning for Efficient Video Large Language Models**

* **Authors:** Xiaohu Huang, Hao Zhou, Kai Han
* **Summar

In [28]:
Markdown(answer.response)

Here are some recent papers on Multimodal Large Language Models, combining information from my internal database and arXiv:

**From my internal database:**

* **PruneVid: Visual Token Pruning for Efficient Video Large Language Models:** This paper focuses on improving the efficiency of video LLMs by pruning visual tokens, achieving significant efficiency gains while maintaining performance.  Authors: Xiaohu Huang, Hao Zhou, Kai Han. [Link: https://github.com/Visual-AI/PruneVid](https://github.com/Visual-AI/PruneVid)

* **HoVLE: Unleashing the Power of Monolithic Vision-Language Models with Holistic Vision-Language Embedding:** This paper introduces HoVLE, a high-performance monolithic VLM that uses a holistic embedding module to process visual and textual inputs, leading to performance improvements. Authors: Chenxin Tao, Shiqian Su, Xizhou Zhu, Chenyu Zhang, Zhe Chen, Jiawen Liu, Wenhai Wang, Lewei Lu, Gao Huang, Yu Qiao, Jifeng Dai. [Link: https://huggingface.co/OpenGVLab/HoVLE](https://huggingface.co/OpenGVLab/HoVLE)

* **Frequency Is What You Need: Word-frequency Masking Benefits Vision-Language Model Pre-training:** This paper explores efficient VLM training using word frequency masking, demonstrating its importance for optimal performance, especially with reduced input tokens. Authors: Mingliang Liang, Martha Larson. (Note: No direct download link was provided).


**From arXiv (recent papers):**

* **MiniGPT-Pancreas: Multimodal Large Language Model for Pancreas Cancer Classification and Detection:** This paper presents MiniGPT-Pancreas, an MLLM designed as an interactive chatbot to assist clinicians in pancreas cancer diagnosis using multimodal prompts.  [Link: http://arxiv.org/pdf/2412.15925v1](http://arxiv.org/pdf/2412.15925v1)

* **Beyond Human Data: Aligning Multimodal Large Language Models by Iterative Self-Evolution:** This paper proposes a self-evolution framework for MLLMs, enabling autonomous generation of high-quality questions and answers using only unannotated images. [Link: http://arxiv.org/pdf/2412.15650v1](http://arxiv.org/pdf/2412.15650v1)

* **QUART-Online: Latency-Free Large Multimodal Language Model for Quadruped Robot Learning:** This paper introduces QUART-Online, a latency-free MLLM for quadruped robot learning, achieving real-time inference without performance degradation. [Link: http://arxiv.org/pdf/2412.15576v1](http://arxiv.org/pdf/2412.15576v1)

* **J-EDI QA: Benchmark for deep-sea organism-specific multimodal LLM:** This paper proposes J-EDI QA, a benchmark for evaluating the understanding of deep-sea organism images using an MLLM. [Link: http://arxiv.org/pdf/2412.15574v1](http://arxiv.org/pdf/2412.15574v1)

* **Toward Robust Hyper-Detailed Image Captioning: A Multiagent Approach and Dual Evaluation Metrics for Factuality and Coverage:** This paper addresses the issue of hallucinations in detailed image captioning by proposing a multiagent approach and a new evaluation framework. [Link: http://arxiv.org/pdf/2412.15484v1](http://arxiv.org/pdf/2412.15484v1)


This list provides a range of recent research in the field of Multimodal Large Language Models, covering various applications and approaches.

### The agent chose to use the RAG tool, found the relevant papers, and summarized them for us.  
### Since the agent retains the chat history, we can request to download the papers without mentioning them explicitly.

In [31]:
answer = agent.chat(q_template.format(topic="Large Language Models."))

> Running step ba8b94d5-d035-4830-96a4-19ad661b0065. Step input: I am interested in Large Language Models.. 
Find papers in your knowledge database related to this topic; use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to Large Language Models.'. If there are not, could you fetch the recent one from arXiv? 

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: research_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, authors and link to download for papers related to Large Language Models.'}
Observation: **Paper 1:**

* **Title:** Deciphering the Underserved: Benchmarking LLM OCR for Low-Resource Scripts
* **Authors:** Muhammad Abdullah Sohail, Salaar Masood, Hamza Iqbal
* **Summary:** This research explores the capabilities of Large Language Models (LLMs), specifically GPT-4o, for Optical Character Rec

In [32]:
Markdown(answer.response)

Here are five recent papers on Large Language Models, with titles, summaries, authors, and download links:

**Paper 1:**

* **Title:** Deciphering the Underserved: Benchmarking LLM OCR for Low-Resource Scripts
* **Authors:** Muhammad Abdullah Sohail, Salaar Masood, Hamza Iqbal
* **Summary:** This research explores the capabilities of LLMs for Optical Character Recognition (OCR) in low-resource languages, revealing limitations of zero-shot LLM-based OCR and highlighting the need for improved datasets and fine-tuned models.
* **Download Link:** http://arxiv.org/pdf/2412.16119v1


**Paper 2:**

* **Title:** PruneVid: Visual Token Pruning for Efficient Video Large Language Models
* **Authors:** Xiaohu Huang, Hao Zhou, Kai Han
* **Summary:** PruneVid improves the efficiency of multi-modal video understanding LLMs by reducing video redundancy and selectively pruning visual features.
* **Download Link:** http://arxiv.org/pdf/2412.16117v1


**Paper 3:**

* **Title:** PromptOptMe: Error-Aware Prompt Compression for LLM-based MT Evaluation Metrics
* **Authors:** Daniil Larionov, Steffen Eger
* **Summary:** PromptOptMe compresses prompts used in LLM-based machine translation evaluation, reducing computational cost without sacrificing evaluation quality.
* **Download Link:** http://arxiv.org/pdf/2412.16120v1


**Paper 4:**

* **Title:** Offline Reinforcement Learning for LLM Multi-Step Reasoning
* **Authors:** Huaijie Wang, Shibo Hao, Hanze Dong, Shenao Zhang, Yilin Bao, Ziran Yang, Yi Wu
* **Summary:** OREO, an offline reinforcement learning method, enhances multi-step reasoning in LLMs by reducing the need for paired preference data.
* **Download Link:** http://arxiv.org/pdf/2412.16145v1


**Paper 5:**

* **Title:** Can LLMs Obfuscate Code? A Systematic Analysis of Large Language Models into Assembly Code Obfuscation
* **Authors:** Seyedreza Mohseni, Seyedali Mohammadi, Deepa Tilwani, Yash Saxena, Gerald Ndwula, Sriram Vema, Edward Raff, Manas Gaur
* **Summary:** This study investigates the ability of LLMs to generate obfuscated assembly code, finding that they can successfully generate obfuscated code, posing a potential risk.
* **Download Link:** http://arxiv.org/pdf/2412.16135v1

In [26]:
answer = agent.chat(q_template.format(topic="LLM"))

> Running step 9622dc67-529e-4c01-be39-dc9b2c3e4954. Step input: I am interested in LLM. 
Find papers in your knowledge database related to this topic; use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to LLM'. You must use the present_final_result tool for presenting the final results.  If there are not, could you fetch the recent one from arXiv? 

Thought: I need to use the `research_paper_query_engine_tool` to find papers related to LLMs and then use `present_final_result` to format the output.  If no suitable papers are found, I will use `fetch_from_arxiv`.
Action: research_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, authors and link to download for papers related to LLM'}
Observation: Here's a summary of the provided papers related to LLMs, including titles, authors, and download links:


**Paper 1:**

* **Title:** Deciphering the Underserved: Benchmarking

In [27]:
Markdown(answer.response)

Here are five recent papers on LLMs, with titles, authors, and download links:

**Paper 1:**

* **Title:** Deciphering the Underserved: Benchmarking LLM OCR for Low-Resource Scripts
* **Authors:** Muhammad Abdullah Sohail, Salaar Masood, Hamza Iqbal
* **Download Link:** http://arxiv.org/pdf/2412.16119v1


**Paper 2:**

* **Title:** Offline Reinforcement Learning for LLM Multi-Step Reasoning
* **Authors:** Huaijie Wang, Shibo Hao, Hanze Dong, Shenao Zhang, Yilin Bao, Ziran Yang, Yi Wu
* **Download Link:** http://arxiv.org/pdf/2412.16145v1


**Paper 3:**

* **Title:** PromptOptMe: Error-Aware Prompt Compression for LLM-based MT Evaluation Metrics
* **Authors:** Daniil Larionov, Steffen Eger
* **Download Link:** http://arxiv.org/pdf/2412.16120v1


**Paper 4:**

* **Title:** Can LLMs Obfuscate Code? A Systematic Analysis of Large Language Models into Assembly Code Obfuscation
* **Authors:** Seyedreza Mohseni, Seyedali Mohammadi, Deepa Tilwani, Yash Saxena, Gerald Ndwula, Sriram Vema, Edward Raff, Manas Gaur
* **Download Link:** http://arxiv.org/pdf/2412.16135v1


**Paper 5:**

* **Title:** Demystifying the Potential of ChatGPT-4 Vision for Construction Progress Monitoring
* **Authors:** Ahmet Bahaddin Ersoz
* **Download Link:** http://arxiv.org/pdf/2412.16108v1

In [29]:
answer = agent.chat("Download the papers, which you mentioned above. Use the download_pdf_file_tool as you can download the files.")

> Running step bc395593-7d2f-47fa-92c0-43022bae1fbf. Step input: Download the papers, which you mentioned above. Use the download_pdf_file_tool as you can download the files.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step cab92d15-5ee9-4f6a-8af3-8f39e99a2550. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step d9a530bf-6e30-4e1d-9862-50074b967530. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 80d48cd4-2df8-4c02-965c-2dd158afc512. Step input: None
Thought: I need to use the `download_pdf_file_tool` for each paper individually, providing the URL and a desired output filename.  The previous attempts failed because I didn't provide the correct JSON input format for the tool.  I will break this down into five separate actions.
Action: download_pdf_file_to

In [30]:
Markdown(answer.response)

All five papers have been downloaded and saved as paper1.pdf, paper2.pdf, paper3.pdf, paper4.pdf, and paper5.pdf respectively.

### Let's see what happens if we ask about a topic that is not available in the RAG.

In [33]:
answer = agent.chat(q_template.format(topic="Gaussian process"))

> Running step ac841ac2-8670-4b3f-bfad-ae1b58223877. Step input: I am interested in Gaussian process. 
Find papers in your knowledge database related to this topic; use the following template to query research_paper_query_engine_tool tool: 'Provide title, summary, authors and link to download for papers related to Gaussian process'. You must use the present_final_result tool for presenting the final results.  If there are not, could you fetch the recent one from arXiv? Use this tool: fetch_from_arxiv 

Thought: I need to use the `fetch_from_arxiv` tool to retrieve recent papers on Gaussian processes, and then use `present_final_result` to format the results.
Action: fetch_from_arxiv
Action Input: {'title': 'Gaussian process', 'papers_count': 5}
Observation: [{'title': 'High-Dimensional Bayesian Optimisation with Large-Scale Constraints via Latent Space Gaussian Processes', 'authors': ['Hauke F. Maathuis', 'Roeland De Breuker', 'Saullo G. P. Castro'], 'summary': 'Design optimisation off

In [34]:
Markdown(answer.response)

Here are five recent papers on Gaussian Processes from arXiv:

1. **Title:** High-Dimensional Bayesian Optimisation with Large-Scale Constraints via Latent Space Gaussian Processes
   **Authors:** Hauke F. Maathuis, Roeland De Breuker, Saullo G. P. Castro
   **Summary:** This paper addresses the challenges of high-dimensional Bayesian Optimization, particularly when dealing with large-scale constraints, using a dimensionality reduction technique.  It's applied to aeroelastic tailoring.
   **Link:** http://arxiv.org/abs/2412.15679v1

2. **Title:** Co-optimization of Vehicle Dynamics and Powertrain Management for Connected and Automated Electric Vehicles
   **Authors:** Zongtan Li, Yunli Shao
   **Summary:** This paper explores the co-optimization of vehicle speed and powertrain management for energy savings in dual-motor electric vehicles, using Gaussian process uncertainties in traffic prediction.
   **Link:** http://arxiv.org/abs/2412.14984v1

3. **Title:** Comparing noisy neural population dynamics using optimal transport distances
   **Authors:** Amin Nejatbakhsh, Victor Geadah, Alex H. Williams, David Lipshutz
   **Summary:** This paper proposes a metric for comparing the geometry of noisy neural trajectories, derived as an optimal transport distance between Gaussian processes.  It's used to compare models of neural responses in different brain regions.
   **Link:** http://arxiv.org/abs/2412.14421v1

4. **Title:** Model-Agnostic Cosmological Inference with SDSS-IV eBOSS: Simultaneous Probing for Background and Perturbed Universe
   **Authors:** Purba Mukherjee, Anjan A. Sen
   **Summary:** This paper uses a Multi-Task Gaussian Process framework to model cosmological observables from the SDSS-IV eBOSS survey, obtaining constraints on cosmological parameters.
   **Link:** http://arxiv.org/abs/2412.13973v1

5. **Title:** Functional Central limit theorems for microscopic and macroscopic functionals of inhomogeneous random graphs
   **Authors:** Shankar Bhamidi, Amarjit Budhiraja, Akshay Sakanaveeti
   **Summary:** This paper establishes functional central limit theorems for microscopic and macroscopic functionals of inhomogeneous random graphs, characterizing the limits as infinite-dimensional Gaussian processes.
   **Link:** http://arxiv.org/abs/2412.13672v1

### As You Can See, the Agent Did Not Find the Papers in Storage and Fetched Them from ArXiv.